## Création de Pipelines avec Sklearn 

Un pipeline en machine learning est un outil structurant permettant d'automatiser et de simplifier le flux de traitement des données et de modélisation. Il consiste en une séquence ordonnée d'étapes, où chaque étape correspond à une transformation spécifique (prétraitement, sélection de caractéristiques, etc.) ou à un modèle d'apprentissage. L'objectif principal d'un pipeline est de garantir un traitement cohérent des données, depuis leur préparation jusqu'à l'entraînement et la prédiction, en minimisant le risque d'erreurs manuelles. En encapsulant toutes les étapes dans une seule entité, un pipeline favorise la réplicabilité, améliore la lisibilité du code et facilite la validation croisée sur des processus complets. Il est donc crucial en machine learning pour développer des modèles robustes et maintenables, surtout dans des environnements où la préparation des données et l'entraînement des modèles nécessitent une grande cohérence et rigueur.

Dans ce notebook, je présente tour à tour les étapes de construction d'un pipeline.

### Importation des librairies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.linear_model import SGDClassifier
import seaborn as sns 
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.preprocessing import (LabelEncoder, OrdinalEncoder, OneHotEncoder,
                                    LabelBinarizer, MultiLabelBinarizer)
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

### Importation du data set 

In [52]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Pour effectuer notre preprocessing on a souvent besoin d'effectuer plusieurs opérations telles que l'encodage, la standardisation, la normalisation, le traitement des outliers, des valeurs manquantes, etc. Toutes ces étapes respectent une séquence d'opérations précises. A chaque opération est souvent associée un transformer (StandarScaler, OneHotEncoder, etc.). L'intérêt de la construction d'un pipeline est qu'il permet de regrouper toutes ces opérations de façon ordonner à travers des fonctions bien définies.

Pour développer une pipeline on peut utiliser la fonction make_pipeline() du module sklearn.pipeline. Mais pour traiter des data sets hétérogènes (avec un mélange de type de variables : continues, discrètes, strings...) il faut utiliser des fonctions un peu plus techniques.

make_column_transformer() permet ainsi de créer un transformer qui ne s'applique que sur certaines colonnes de votre dataset. Il est souvent utilisé pour traiter les variables numériques et les variables catégorielles de façon différente.

make_column_selector() est une fonctionnalité de sklearn qui permet de séléctionner les colonnes d'un dataset selon leur dtype.

La fonction make_union permet de construire des pipelines paralleles, dont les résultats sont concaténé dans un tableau final. Cette fonction existe également sous forme de Classe avec FeatureUnion.

In [53]:
# Par exemple en précisant les noms des variables 
y = titanic['survived']
X = titanic.drop('survived', axis=1)

In [11]:
from sklearn.compose import make_column_transformer

In [13]:
# Creation d'un transformer qui prend les opérations de transformation 

transformer = make_column_transformer((StandardScaler(),
                                      ['age', 'fare']))

In [ ]:
# On peut l'entrainer avec fit_transform 
# En lui passant X il va tenir compte uniquement des colonnes sélectionnées
transformer.fit_transform(X)


array([[-0.53037664, -0.50244517],
       [ 0.57183099,  0.78684529],
       [-0.25482473, -0.48885426],
       ...,
       [        nan, -0.17626324],
       [-0.25482473, -0.04438104],
       [ 0.15850313, -0.49237783]])

In [ ]:
# On va passer nos transformer dans nos modèles pour les utlises de façon correcte 

#model = make_pipeline(transformer, SGDClassifier())

In [30]:
# Spearation des types de variables 

numerical_features = ['pclass', 'age', 'fare']
categorical_features = ['sex', 'deck', 'alone']

In [31]:
# Creation des types de pipeline 

numerical_pipeline = make_pipeline(SimpleImputer(),
                                    StandardScaler())
categorical_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                     OneHotEncoder())

In [32]:
# On obtient donc un transformer
preprocessor = make_column_transformer((numerical_pipeline, numerical_features),
                        (categorical_pipeline,categorical_features))

In [ ]:
# On cree notre pipeline 

model = make_pipeline(preprocessor, SGDClassifier())
model.fit(X, y) 

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'deck', 'alone'])])),
                ('sgdclassifier', SGDClassifier())])

#### Faire un pipeline sans préciser les noms des variables mais plutôt le type. 

Très utile si on a de nombreuses colonnes. 

In [35]:
from sklearn.compose import make_column_selector

In [36]:
# Spearation des types de variables 

numerical_features = make_column_selector(dtype_include=np.number)
categorical_features = make_column_selector(dtype_exclude=np.number)

In [37]:
# On cree notre pipeline 

model = make_pipeline(preprocessor, SGDClassifier())
model.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['pclass', 'age', 'fare']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  ['sex', 'deck', 'alone'])])),
                ('sgdclassifier', SGDClassifier())])

### Creation de pipelines parelleles avec make_union

In [42]:
from sklearn.pipeline import make_union
from sklearn.preprocessing import StandardScaler, Binarizer

In [76]:
# Spearation des types de variables 

imputer = SimpleImputer(strategy='mean')
X['age']= imputer.fit_transform(X[['age']])
numerical_features = X[['age', 'fare']]


In [77]:
pipeline = make_union(StandardScaler(), Binarizer())

In [81]:
result = pipeline.fit_transform(numerical_features)

In [82]:
result.shape 

(891, 4)